## Exercise 1.1

Detect the cars using background subtraction and frame differencing using OpenCV

In [ ]:
%pip install opencv-python
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import cv2
import numpy as np


def clean_foreground_mask(raw_mask):
    _, binary = cv2.threshold(raw_mask, 100, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    return cleaned


def detect_main_street_cars(fg_mask, frame_shape, min_blob_area=4000):
    contours, _ = cv2.findContours(
        fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    contours = [c for c in contours if cv2.contourArea(c) > min_blob_area]

    frame_h, frame_w = frame_shape[:2]

    # only lower half (main street)
    contours = [c for c in contours if cv2.boundingRect(c)[1] > frame_h / 2]

    contours = [cv2.convexHull(c) for c in contours]

    boxes = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        boxes.append((x, y, w, h))

    return boxes


def run_task_11(video_path="Traffic_Laramie_1.mp4"):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    subtractor = cv2.createBackgroundSubtractorKNN(
        detectShadows=False, history=10000, dist2Threshold=900
    )

    while True:
        ok, frame = video.read()
        if not ok:
            break

        blurred = cv2.GaussianBlur(frame, (5, 5), 0)

        raw_mask = subtractor.apply(blurred)
        fg_mask = clean_foreground_mask(raw_mask)

        boxes = detect_main_street_cars(fg_mask, frame.shape, min_blob_area=4000)

        frame_h, frame_w = frame.shape[:2]
        cv2.line(frame, (0, int(frame_h * 0.50)), (frame_w, int(frame_h * 0.50)), (0, 255, 255), 2)

        for (x, y, w, h) in boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("task 1.1 - tracking", frame)
        cv2.imshow("task 1.1 - fg mask", fg_mask)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video.release()
    cv2.destroyAllWindows()


run_task_11("Traffic_Laramie_1.mp4")